In [57]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

In [58]:
df = pd.read_csv("Heart_disease_cleveland.csv")

df, df_validate = np.split(df.sample(frac=1, random_state=0), [int(.8*len(df))])

c:\_Studium\Proseminar\Practical\venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Train / Test Split

In [59]:
def create_features(df):
    df = df.copy()

    features = [
        "age","sex","cp","trestbps","chol","fbs","restecg",
        "thalach","exang","oldpeak","slope","ca","thal"
    ]

    X = df[features]
    y = df["target"]

    return X, y

### Create Model

In [60]:
def get_model():
    estimator = DecisionTreeClassifier(
        random_state=0,
        max_depth=3
    )
    
    model = AdaBoostClassifier(
        learning_rate=0.1,
        n_estimators=12,
        random_state=0
    )
    
    return model

In [61]:
kf = KFold(n_splits=10, shuffle=True, random_state=0)
scores = []

for train_index, test_index in kf.split(df):
    train = df.iloc[train_index]
    test = df.iloc[test_index]

    X_train, y_train = create_features(df=train)
    X_test, y_test = create_features(df=test)

    model = get_model()

    model.fit(X_train, y_train)
    
    score = model.score(X_test, y_test)
    scores.append(score)

print(np.average(scores))
print(scores)

0.8394999999999999
[0.76, 0.76, 0.875, 0.875, 0.875, 0.8333333333333334, 0.8333333333333334, 0.9166666666666666, 0.8333333333333334, 0.8333333333333334]


### Validation

In [62]:
X, y = create_features(df=df)
X_val, y_val = create_features(df=df_validate)

model = get_model()

model.fit(X, y)

model.score(X_val, y_val)

0.8524590163934426